In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from dataUtils import DataUtils as utils
from model_utils import ModelUtils as mutls

C:\Users\Yang\.conda\envs\test_speedcom\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


In [2]:
from keras import Sequential
from keras.layers import Dense, Embedding, LSTM, GRU, Conv1D, Activation

Using TensorFlow backend.


In [3]:
df=pd.read_csv('temp_cfprint.tsv', sep='\t')

In [4]:
df.head()

,#,Name,name_smiles,Wavelength,Epsilon,Quantum Yield,cfp
0,1,Benzene,C1=CC=CC=C1,254.75,210,0.053,"[224, 255, 255, 255, 0, 8, 0, 0, 3, 0, 0, 0, 2..."
1,2,Toluene,CC1=CC=CC=C1,261.75,2860,0.170,"[224, 255, 255, 255, 0, 8, 0, 0, 11, 0, 0, 0, ..."
2,3,o-Xylene,CC1=CC=CC=C1C,263.00,254,0.170,"[224, 255, 255, 255, 0, 8, 0, 0, 10, 0, 0, 0, ..."
3,4,m-Xylene,CC1=CC(=CC=C1)C,265.00,284,0.130,"[224, 255, 255, 255, 0, 8, 0, 0, 12, 0, 0, 0, ..."
4,5,p-Xylene,CC1=CC=C(C=C1)C,275.00,770,0.220,"[224, 255, 255, 255, 0, 8, 0, 0, 8, 0, 0, 0, 1..."


In [5]:
X_fp=utils.finger_print_clean(df=df,colname_string_array='cfp')

fingerprint column shape is (267, 137)


In [6]:
X_fp.shape

(267, 137)

In [7]:
Y = df.values[:,3]

### First model original Y

In [8]:
d_cat = mutls.combine_columns((X_fp, Y))

In [9]:
X, y = d_cat[:,0:-1], d_cat[:,-1]

In [10]:
y

array([254.75, 261.75, 263.0, 265.0, 275.0, 265.0, 278.0, 271.0, 271.0,
       270.75, 220.0, 222.0, 242.0, 227.0, 241.0, 288.0, 251.0, 279.0,
       265.0, 228.0, 258.0, 248.0, 236.0, 274.0, 277.0, 227.0, 222.0,
       218.0, 310.25, 291.0, 240.0, 251.0, 263.0, 245.0, 302.0, 252.0,
       283.0, 225.0, 316.0, 230.0, 442.0, 247.5, 276.25, 294.75, 312.0,
       251.0, 206.0, 209.0, 212.0, 245.0, 223.0, 278.0, 275.0, 356.25,
       475.0, 578.0, 252.0, 241.0, 258.0, 372.5, 270.25, 303.0, 228.0,
       239.0, 245.0, 218.0, 227.0, 235.0, 278.75, 276.0, 293.75, 326.0,
       330.0, 353.25, 432.25, 425.0, 372.0, 451.0, 251.0, 276.0, 227.0,
       257.0, 265.0, 270.0, 215.0, 281.0, 238.0, 204.0, 219.0, 245.0,
       217.0, 253.0, 231.75, 206.0, 278.0, 502.0, 413.0, 286.0, 336.0,
       460.5, 219.0, 413.0, 247.0, 349.0, 262.0, 257.5, 274.25, 211.0,
       278.0, 261.0, 243.0, 266.75, 263.75, 258.25, 228.0, 259.0, 212.0,
       263.0, 265.0, 206.0, 248.0, 281.0, 239.75, 225.0, 288.0, 211.0,
  

In [11]:
X_train, X_test, y_train, y_test = utils.splitData(X, y, ratio=0.20)
y_train = np.reshape(y_train, (-1, 1))
y_test = np.reshape(y_test, (-1, 1))
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(213, 137)
(54, 137)
(213, 1)
(54, 1)


In [12]:
unique_int_counts = len(np.unique(X_train))

In [13]:
np.unique(X_train)

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18,
       19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35,
       36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52,
       53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69,
       70, 72, 73, 74, 75, 76, 77, 78, 80, 81, 82, 84, 85, 86, 88, 89, 90,
       92, 93, 94, 95, 96, 97, 98, 100, 101, 102, 104, 105, 106, 108, 109,
       110, 112, 113, 114, 116, 117, 118, 120, 121, 122, 124, 125, 126,
       128, 129, 130, 132, 133, 134, 136, 137, 138, 140, 141, 142, 144,
       145, 146, 148, 149, 150, 152, 153, 154, 156, 157, 158, 160, 161,
       162, 164, 165, 166, 168, 169, 170, 172, 173, 174, 176, 177, 178,
       180, 181, 182, 184, 185, 186, 188, 189, 190, 192, 193, 194, 196,
       197, 198, 200, 201, 202, 204, 205, 206, 208, 209, 210, 212, 213,
       214, 216, 217, 218, 220, 221, 222, 224, 225, 226, 228, 229, 230,
       232, 233, 234, 236, 237, 238, 240, 241, 2

In [14]:
unique_int_counts

212

In [15]:
X_train.shape

(213, 137)

In [16]:
from keras.models import Sequential
from keras.layers import LSTM, Dense, Embedding, Dropout, Flatten

In [17]:
hidden_size = 50
model = Sequential()
model.add(Embedding(256, hidden_size, input_length= X.shape[1]))
model.add(LSTM(hidden_size, return_sequences=True))
model.add(LSTM(hidden_size, return_sequences=True))
model.add(LSTM(hidden_size, return_sequences=True))
model.add(Dropout(0.4))
model.add(Dense(hidden_size, activation='relu'))
model.add(Flatten())
model.add(Dense(1, activation='linear'))
model.compile(loss='mean_squared_error', optimizer='Adam', metrics=['mean_absolute_error'])
model.summary() ##use mean_squared_error loss

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 137, 50)           12800     
_________________________________________________________________
lstm_1 (LSTM)                (None, 137, 50)           20200     
_________________________________________________________________
lstm_2 (LSTM)                (None, 137, 50)           20200     
_________________________________________________________________
lstm_3 (LSTM)                (None, 137, 50)           20200     
_________________________________________________________________
dropout_1 (Dropout)          (None, 137, 50)           0         
_________________________________________________________________
dense_1 (Dense)      

In [18]:
model.fit(X_train, y_train, batch_size=32, validation_data=(X_test, y_test), epochs=100)

Instructions for updating:
Use tf.cast instead.
Train on 213 samples, validate on 54 samples
Epoch 1/100
213/213 [==============================] - 6s 29ms/step - loss: 160379.1317 - mean_absolute_error: 371.3096 - val_loss: 134951.5286 - val_mean_absolute_error: 345.1137
Epoch 2/100
213/213 [==============================] - 2s 10ms/step - loss: 152943.6567 - mean_absolute_error: 360.9150 - val_loss: 113847.8487 - val_mean_absolute_error: 312.9310
Epoch 3/100
213/213 [==============================] - 2s 10ms/step - loss: 115158.2079 - mean_absolute_error: 304.2075 - val_loss: 69818.1665 - val_mean_absolute_error: 232.3322
Epoch 4/100
213/213 [==============================] - 2s 10ms/step - loss: 63348.9899 - mean_absolute_error: 198.0327 - val_loss: 23348.1945 - val_mean_absolute_error: 100.9977
Epoch 5/100
213/213 [==============================] - 2s 10ms/step - loss: 27028.3849 - mean_absolute_error: 132.7507 - val_loss: 23013.9338 - val_mean_absolute_error: 136.4390
Epoch 6/100


213/213 [==============================] - 2s 10ms/step - loss: 22515.2376 - mean_absolute_error: 129.4225 - val_loss: 16340.0175 - val_mean_absolute_error: 115.1471
Epoch 46/100
213/213 [==============================] - 2s 10ms/step - loss: 22502.7993 - mean_absolute_error: 130.4169 - val_loss: 16153.9757 - val_mean_absolute_error: 113.7870
Epoch 47/100
213/213 [==============================] - 2s 10ms/step - loss: 22580.9640 - mean_absolute_error: 129.9798 - val_loss: 16040.5360 - val_mean_absolute_error: 112.7511
Epoch 48/100
213/213 [==============================] - 2s 10ms/step - loss: 22923.9164 - mean_absolute_error: 128.6792 - val_loss: 16017.2708 - val_mean_absolute_error: 112.5095
Epoch 49/100
213/213 [==============================] - 2s 10ms/step - loss: 22583.5235 - mean_absolute_error: 130.2565 - val_loss: 16409.9561 - val_mean_absolute_error: 115.5917
Epoch 50/100
213/213 [==============================] - 2s 10ms/step - loss: 22590.3316 - mean_absolute_error: 130.143

Epoch 91/100
213/213 [==============================] - 2s 10ms/step - loss: 22660.0361 - mean_absolute_error: 129.4809 - val_loss: 16407.8465 - val_mean_absolute_error: 115.5797
Epoch 92/100
213/213 [==============================] - 2s 10ms/step - loss: 22878.2242 - mean_absolute_error: 131.8275 - val_loss: 16200.1528 - val_mean_absolute_error: 114.1569
Epoch 93/100
213/213 [==============================] - 2s 10ms/step - loss: 22554.7737 - mean_absolute_error: 128.8141 - val_loss: 15847.5324 - val_mean_absolute_error: 110.1059
Epoch 94/100
213/213 [==============================] - 2s 10ms/step - loss: 22663.6154 - mean_absolute_error: 128.5580 - val_loss: 16190.2944 - val_mean_absolute_error: 114.0807
Epoch 95/100
213/213 [==============================] - 2s 11ms/step - loss: 22755.8214 - mean_absolute_error: 131.5070 - val_loss: 16172.7291 - val_mean_absolute_error: 113.9420
Epoch 96/100
213/213 [==============================] - 3s 12ms/step - loss: 22629.8180 - mean_absolute_e

### Second model, category model

In [19]:
Y_category = mutls.get_y_category(y_actual=Y, min_edge=200, bin_width=20)

In [20]:
cls_counts = mutls.get_class_count(Y_category)

In [21]:

print(cls_counts)

{0.0: 18, 1.0: 23, 2.0: 40, 3.0: 44, 4.0: 14, 5.0: 7, 6.0: 8, 7.0: 4, 8.0: 6, 9.0: 3, 10.0: 12, 11.0: 10, 12.0: 7, 13.0: 4, 14.0: 7, 15.0: 10, 16.0: 5, 17.0: 10, 18.0: 6, 19.0: 6, 20.0: 6, 21.0: 3, 22.0: 4, 23.0: 2, 24.0: 2, 25.0: 2, 28.0: 1, 29.0: 2, 30.0: 1}


In [22]:
X_fp.shape

(267, 137)

In [23]:
d_cat = mutls.combine_columns((X_fp, Y_category))

In [24]:
subsampled=mutls.subsampling(d_cat,-1,15,cls_counts)

In [25]:
subsampled

array([[224, 255, 255, ..., 0, 0, 2.0],
       [224, 255, 255, ..., 0, 0, 3.0],
       [224, 255, 255, ..., 0, 0, 1.0],
       ...,
       [224, 255, 255, ..., 0, 0, 10.0],
       [224, 255, 255, ..., 0, 0, 10.0],
       [224, 255, 255, ..., 0, 0, 29.0]], dtype=object)

In [26]:
X, y = subsampled[:,0:-1], subsampled[:,-1]
y_onehot = mutls.onehot_encode_y(y=y, num_class=31)
X_train, X_test, y_train, y_test = utils.splitData(X, y_onehot, ratio=0.10)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(181, 137)
(21, 137)
(181, 31)
(21, 31)


### 2nd fingerprint model RNN LSTM with category Y

In [27]:
X_train = X_train.reshape((X_train.shape[0],X_train.shape[1],1))
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

rnn1_size = 224
rnn2_size = 384
dropout_rate = 0.2
loss = 'categorical_crossentropy'
metrics = ['accuracy']
num_classes = 31
optimizer = 'adam' #use adam optimizer

model_1 = Sequential()
model_1.add(LSTM(rnn1_size, return_sequences=True, input_shape=(137,1))) #137
model_1.add(LSTM(rnn2_size))
model_1.add(Dense(num_classes, activation='softmax'))
model_1.compile(optimizer=optimizer, loss=loss, metrics=metrics)
model_1.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_4 (LSTM)                (None, 137, 224)          202496    
_________________________________________________________________
lstm_5 (LSTM)                (None, 384)               935424    
_________________________________________________________________
dense_3 (Dense)              (None, 31)                11935     
Total params: 1,149,855
Trainable params: 1,149,855
Non-trainable params: 0
_________________________________________________________________


In [28]:
batch_size = 16
epochs = 100

model_1.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, validation_data=[X_test, y_test])

Train on 181 samples, validate on 21 samples
Epoch 1/100
181/181 [==============================] - 13s 70ms/step - loss: 3.4299 - acc: 0.0552 - val_loss: 3.3206 - val_acc: 0.0000e+00
Epoch 2/100
181/181 [==============================] - 9s 50ms/step - loss: 3.2880 - acc: 0.0884 - val_loss: 3.2661 - val_acc: 0.0476
Epoch 3/100
181/181 [==============================] - 9s 51ms/step - loss: 3.2250 - acc: 0.0829 - val_loss: 3.2303 - val_acc: 0.0952
Epoch 4/100
181/181 [==============================] - 9s 51ms/step - loss: 3.1973 - acc: 0.1050 - val_loss: 3.1175 - val_acc: 0.0952
Epoch 5/100
181/181 [==============================] - 9s 50ms/step - loss: 3.1884 - acc: 0.0608 - val_loss: 3.1575 - val_acc: 0.0000e+00
Epoch 6/100
181/181 [==============================] - 9s 51ms/step - loss: 3.1696 - acc: 0.0718 - val_loss: 3.1519 - val_acc: 0.0952
Epoch 7/100
181/181 [==============================] - 9s 51ms/step - loss: 3.2686 - acc: 0.0994 - val_loss: 3.0736 - val_acc: 0.0952
Epoch 8/

Training acc 0.25, testing accuracy 0.05 (max 0.15)